In [1]:
#!export WANDB_API_KEY=2a631ea744b03506a1330798e0724d1d917a821f

In [2]:
%env WANDB_API_KEY=2a631ea744b03506a1330798e0724d1d917a821f

env: WANDB_API_KEY=2a631ea744b03506a1330798e0724d1d917a821f


In [3]:
# 모듈 import

import sys
sys.path.append("../mmdetection/")

from mmengine.hooks import Hook
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.registry import DATASETS
from mmdet.utils import register_all_modules

In [4]:
import wandb
wandb.login()
from wandb_custom_hooks import WandbInitHook

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mevest71 (mevest71-boostcamp). Use `wandb login --relogin` to force relogin


In [5]:
# 모든 모듈 등록
register_all_modules()

In [ ]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('../mmdetection/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')

root='./dataset/'

# dataset config 수정
cfg.dataset_type = 'CocoDataset'
cfg.data_root = root

# Train dataset config 수정
cfg.train_dataloader = dict(
    batch_size=4,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        ann_file='train.json',
        data_prefix=dict(img=''),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=cfg.train_pipeline,
        metainfo=dict(classes=classes)
    )
)

# Validation dataset config 수정
cfg.val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        ann_file='test.json',
        data_prefix=dict(img=''),
        test_mode=True,
        pipeline=cfg.test_pipeline,
        metainfo=dict(classes=classes)
    )
)

# Test dataset config 수정 (validation과 동일하게 설정)
cfg.test_dataloader = cfg.val_dataloader

# Train, val, test evaluator 설정
cfg.train_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'train.json',
    metric='bbox',
    format_only=False
)

cfg.val_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'test.json',
    metric='bbox',
    format_only=False
)

cfg.test_evaluator = cfg.val_evaluator

# 기타 설정
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=12, val_interval=1)
cfg.val_cfg = dict(type='ValLoop')
cfg.test_cfg = dict(type='TestLoop')

# 체크포인트 pth와 로그 저장
# logger : interval로 받은 숫자만큼의 iteration마다 로그를 저장함
# checkpoint : max_keep_ckpts의 수만큼 최근 체크 포인트를 저장하고 유지함
cfg.default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    logger=dict(type='LoggerHook', interval=50),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(type='CheckpointHook', interval=1, max_keep_ckpts=3),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='DetVisualizationHook'),
)

cfg.env_cfg = dict(
    cudnn_benchmark=False,
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'),
)

cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10
#cfg.model.bbox_head.num_classes = 10

cfg.optim_wrapper.optimizer.lr = 0.02
cfg.optim_wrapper.clip_grad = dict(max_norm=35, norm_type=2)

# WandB 설정 추가
cfg.visualizer.update(
    type='DetLocalVisualizer',
    vis_backends=[
        dict(type='LocalVisBackend'),
        dict(
            type='WandbVisBackend',
            init_kwargs=dict(
                project='Project2',
                name='----------'
                #entity='my_entity'  # 필요한 경우 추가
            )
        )
    ]
)

# WandB 로거 추가
cfg.log_processor = dict(
    type='LogProcessor',
    window_size=50,
    by_epoch=True,
    custom_cfg=[
        dict(
            data_src='wandb',
            log_name='wandb',
            type='WandbLoggerHook',
            init_kwargs={
                'project': 'Project2'
                #'entity': 'my_entity'  # 필요한 경우 추가
            }
        )
    ]
)


# Custom hooks 추가
cfg.custom_hooks = [
    dict(
        type='wandb_custom_hooks.WandbInitHook',
        project='Project2',
        name='ex_test3'
        )
    ]

# Runner 생성 및 학습 시작
runner = Runner.from_cfg(cfg)

runner.train()


10/14 17:37:04 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 229315886
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code

10/14 17:37:12 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/14 17:37:12 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) WandbInitHook                      
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_i

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


10/14 17:37:18 - mmengine - INFO - load model from: torchvision://resnet50
10/14 17:37:18 - mmengine - INFO - Loads checkpoint by torchvision backend from path: torchvision://resnet50
10/14 17:37:19 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

10/14 17:37:19 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
10/14 17:37:19 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
10/14 17:37:19 - mmengine - INFO - Checkpoints will be saved to /data/ephemeral/home/kjh/level2-objectdetection-cv-16/work_dirs/faster_rcnn_r50_fpn_1x_trash.
10/14 17:38:22 - mmengine - INFO - Epoch(train)  [1][  50/1221]  lr: 1.9820e-03  eta: 5:07:54  time: 1.2652  data_time: 0.0136  memory: 4460  grad_norm: 6.9995  loss: 1.1699  loss_rpn_cls: 0.4538  loss_r

wandb: WARNING Tried to log to step 0 that is less than the current step 1004. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


10/14 18:01:14 - mmengine - INFO - Epoch(val)  [1][  50/4871]    eta: 0:11:36  time: 0.1446  data_time: 0.0054  memory: 4460  
10/14 18:01:21 - mmengine - INFO - Epoch(val)  [1][ 100/4871]    eta: 0:11:06  time: 0.1349  data_time: 0.0022  memory: 697  
10/14 18:01:28 - mmengine - INFO - Epoch(val)  [1][ 150/4871]    eta: 0:10:57  time: 0.1381  data_time: 0.0019  memory: 697  
10/14 18:01:35 - mmengine - INFO - Epoch(val)  [1][ 200/4871]    eta: 0:10:44  time: 0.1344  data_time: 0.0019  memory: 697  
10/14 18:01:41 - mmengine - INFO - Epoch(val)  [1][ 250/4871]    eta: 0:10:36  time: 0.1368  data_time: 0.0022  memory: 697  
10/14 18:01:48 - mmengine - INFO - Epoch(val)  [1][ 300/4871]    eta: 0:10:29  time: 0.1375  data_time: 0.0022  memory: 697  
10/14 18:01:55 - mmengine - INFO - Epoch(val)  [1][ 350/4871]    eta: 0:10:20  time: 0.1343  data_time: 0.0020  memory: 697  
